In [1]:
import numpy as np

In [ ]:
# Speech-to-Text Evaluation
jiwer = "^3.3.1"              # Latest stable version
rapidfuzz = "^3.10.1"         # Latest version with performance improvements

# Diarization Evaluation  
numpy = "^2.2.0"              # Latest stable version
pyannote-audio = "^3.3.2"     # Includes pyannote.metrics
pyannote-core = "^5.0.0"      # Core annotation utilities

# Optional Configuration
pyyaml = "^6.0.2"   

In [2]:
from datasets import load_dataset
import os
from pathlib import Path

# Create local datasets directory
local_data_dir = Path("../datasets/stt/arabic_common_voice")
local_data_dir.mkdir(parents=True, exist_ok=True)

print(f"Downloading dataset to: {local_data_dir.absolute()}")

# Download and cache the dataset locally
# Login using `huggingface-cli login` if you haven't already
try:
    ds = load_dataset(
        "Geethuzzz/common_voice_17_0_arabic_New_cleaned",
        cache_dir=str(local_data_dir / "cache"),
        trust_remote_code=True
    )
    
    print("Dataset downloaded successfully!")
    print(f"Dataset splits: {list(ds.keys())}")
    
    # Show basic info about the dataset
    for split_name, split_data in ds.items():
        print(f"{split_name}: {len(split_data)} samples")
        
except Exception as e:
    print(f"Error downloading dataset: {e}")
    print("Make sure you're logged in with: huggingface-cli login")

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'Geethuzzz/common_voice_17_0_arabic_New_cleaned' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Generating validated split: 100%|██████████| 78156/78156 [00:13<00:00, 5674.96 examples/s]



Dataset downloaded successfully!
Dataset splits: ['train', 'validation', 'test', 'other', 'invalidated', 'validated']
train: 28369 samples
validation: 10470 samples
test: 10480 samples
other: 41628 samples
invalidated: 15108 samples
validated: 78156 samples


In [4]:
# Save dataset splits to local files for evaluation framework
import json
import pandas as pd

if 'ds' in locals() and ds is not None:
    print("Saving dataset splits to local files...")
    
    for split_name, split_data in ds.items():
        # Create directory for this split
        split_dir = local_data_dir / split_name
        split_dir.mkdir(exist_ok=True)
        
        # Convert to pandas for easier handling
        df = split_data.to_pandas()
        
        print(f"\nProcessing {split_name} split:")
        print(f"Columns: {list(df.columns)}")
        print(f"Sample count: {len(df)}")
        
        # Save as parquet for efficient storage
        parquet_file = split_dir / f"{split_name}.parquet"
        df.to_parquet(parquet_file, index=False)
        print(f"Saved to: {parquet_file}")
        
        # Also save first 100 samples as JSON for quick inspection (excluding 'audio')
        sample_file = split_dir / f"{split_name}_sample.json"
        sample_data = df.head(100).drop(columns=['audio'], errors='ignore').to_dict('records')
        with open(sample_file, 'w', encoding='utf-8') as f:
            json.dump(sample_data, f, ensure_ascii=False, indent=2)
        print(f"Sample saved to: {sample_file}")
        
        # If there's audio data, show some info
        if 'audio' in df.columns:
            print(f"Audio column present with {df['audio'].notna().sum()} non-null entries")
        
        # Show first few text samples if available
        text_columns = [col for col in df.columns if 'text' in col.lower() or 'sentence' in col.lower()]
        if text_columns:
            print(f"Text columns found: {text_columns}")
            for col in text_columns[:1]:  # Show first text column
                print(f"Sample {col}: {df[col].iloc[0] if len(df) > 0 else 'N/A'}")
    
    print(f"\nDataset successfully saved to: {local_data_dir.absolute()}")
else:
    print("Dataset not loaded. Run the previous cell first.")

Saving dataset splits to local files...

Processing train split:
Columns: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant']
Sample count: 28369
Saved to: ../datasets/stt/arabic_common_voice/train/train.parquet
Sample saved to: ../datasets/stt/arabic_common_voice/train/train_sample.json
Audio column present with 28369 non-null entries
Text columns found: ['sentence']
Sample sentence: وما كان ربك ليهلك القرى بظلم وأهلها مصلحون

Processing validation split:
Columns: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant']
Sample count: 10470
Saved to: ../datasets/stt/arabic_common_voice/validation/validation.parquet
Sample saved to: ../datasets/stt/arabic_common_voice/validation/validation_sample.json
Audio column present with 10470 non-null entries
Text columns found: ['sentence']
Sample sentence: هو يقرأ كتاباً.

Processing test split:
Colu

In [2]:
from datasets import load_dataset
import os
from pathlib import Path

# Create local datasets directory for LibriSpeech
librispeech_data_dir = Path("../datasets/stt/librispeech_clean")
librispeech_data_dir.mkdir(parents=True, exist_ok=True)

print(f"Downloading LibriSpeech dataset to: {librispeech_data_dir.absolute()}")

# Download and cache the LibriSpeech dataset locally
try:
    librispeech_ds = load_dataset(
        "openslr/librispeech_asr", 
        "clean",
        cache_dir=str(librispeech_data_dir / "cache"),
        trust_remote_code=True
    )
    
    print("LibriSpeech dataset downloaded successfully!")
    print(f"Dataset splits: {list(librispeech_ds.keys())}")
    
    # Show basic info about the dataset
    for split_name, split_data in librispeech_ds.items():
        print(f"{split_name}: {len(split_data)} samples")
        
except Exception as e:
    print(f"Error downloading LibriSpeech dataset: {e}")
    print("Make sure you have internet connection and sufficient disk space")

/workspaces/oryx-cap-upskilling/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'openslr/librispeech_asr' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'openslr/librispeech_asr' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


LibriSpeech dataset downloaded successfully!
Dataset splits: ['test', 'train.100', 'train.360', 'validation']
test: 2620 samples
train.100: 28539 samples
train.360: 104014 samples
validation: 2703 samples
LibriSpeech dataset downloaded successfully!
Dataset splits: ['test', 'train.100', 'train.360', 'validation']
test: 2620 samples
train.100: 28539 samples
train.360: 104014 samples
validation: 2703 samples


In [ ]:
# Save LibriSpeech dataset splits to local files for evaluation framework
import json
import pandas as pd

if 'librispeech_ds' in locals() and librispeech_ds is not None:
    print("Saving LibriSpeech dataset splits to local files...")
    
    for split_name, split_data in librispeech_ds.items():
        # Create directory for this split
        split_dir = librispeech_data_dir / split_name
        split_dir.mkdir(exist_ok=True)
        
        # Convert to pandas for easier handling
        df = split_data.to_pandas()
        
        print(f"\nProcessing {split_name} split:")
        print(f"Columns: {list(df.columns)}")
        print(f"Sample count: {len(df)}")
        
        # Save as parquet for efficient storage
        parquet_file = split_dir / f"{split_name}.parquet"
        df.to_parquet(parquet_file, index=False)
        print(f"Saved to: {parquet_file}")
        
        # Also save first 100 samples as JSON for quick inspection (excluding 'audio')
        sample_file = split_dir / f"{split_name}_sample.json"
        sample_data = df.head(100).drop(columns=['audio'], errors='ignore').to_dict('records')
        with open(sample_file, 'w', encoding='utf-8') as f:
            json.dump(sample_data, f, ensure_ascii=False, indent=2)
        print(f"Sample saved to: {sample_file}")
        
        # If there's audio data, show some info
        if 'audio' in df.columns:
            print(f"Audio column present with {df['audio'].notna().sum()} non-null entries")
            # Show audio sample rate info if available
            if len(df) > 0 and df['audio'].iloc[0] is not None:
                sample_audio = df['audio'].iloc[0]
                if isinstance(sample_audio, dict) and 'sampling_rate' in sample_audio:
                    print(f"Sample rate: {sample_audio['sampling_rate']} Hz")
        
        # Show first few text samples if available
        text_columns = [col for col in df.columns if 'text' in col.lower() or 'sentence' in col.lower()]
        if text_columns:
            print(f"Text columns found: {text_columns}")
            for col in text_columns[:1]:  # Show first text column
                sample_text = df[col].iloc[0] if len(df) > 0 else 'N/A'
                print(f"Sample {col}: {sample_text}")
        
        # Show speaker ID info if available
        if 'speaker_id' in df.columns:
            unique_speakers = df['speaker_id'].nunique()
            print(f"Unique speakers: {unique_speakers}")
    
    print(f"\nLibriSpeech dataset successfully saved to: {librispeech_data_dir.absolute()}")
else:
    print("LibriSpeech dataset not loaded. Run the previous cell first.")

Saving LibriSpeech dataset splits to local files...

Processing test split:
Columns: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id']
Sample count: 2620

Processing test split:
Columns: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id']
Sample count: 2620
Saved to: ../datasets/stt/librispeech_clean/test/test.parquet
Sample saved to: ../datasets/stt/librispeech_clean/test/test_sample.json
Audio column present with 2620 non-null entries
Text columns found: ['text']
Sample text: CONCORD RETURNED TO ITS PLACE AMIDST THE TENTS
Unique speakers: 40
Saved to: ../datasets/stt/librispeech_clean/test/test.parquet
Sample saved to: ../datasets/stt/librispeech_clean/test/test_sample.json
Audio column present with 2620 non-null entries
Text columns found: ['text']
Sample text: CONCORD RETURNED TO ITS PLACE AMIDST THE TENTS
Unique speakers: 40

Processing train.100 split:
Columns: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id']
Sample count: 28539

Processing train.100 

In [ ]:
# Using the Evaluation Framework Dataset Helper
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

# Import evaluation framework components
from config import get_settings
from dataset_manager import (
    list_available_datasets, 
    get_dataset_status,
    setup_eval_datasets,
    download_datasets
)
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load evaluation settings
settings = get_settings()
print(f"📋 Evaluation Framework Configuration Loaded")
print(f"Dataset directory: {settings.datasets.base_download_dir}")
print(f"Cache directory: {settings.datasets.cache_dir}")
print(f"Sample size: {settings.datasets.sample_size}")
print()

# List all available datasets
print("🗂️ Available Datasets:")
print("=" * 60)
available_datasets = list_available_datasets()
for key, info in available_datasets.items():
    print(f"📊 {key}:")
    print(f"    Name: {info['name']}")
    print(f"    Task: {info['task']}")
    print(f"    Language: {info['language']}")
    print(f"    Repository: {info['repo_id']}")
    print(f"    Description: {info['description']}")
    print()

In [ ]:
# Check current dataset status
print("📈 Current Dataset Status:")
print("=" * 50)
status = get_dataset_status(settings)
for key, info in status.items():
    if info['downloaded']:
        print(f"✅ {info['name']} ({key})")
        if 'total_samples' in info:
            print(f"    📊 Total samples: {info['total_samples']:,}")
        if 'splits' in info:
            print(f"    📁 Splits: {', '.join(info['splits'].keys())}")
    else:
        print(f"❌ {info['name']} ({key}) - Not downloaded")
        if 'error' in info:
            print(f"    ⚠️  Error: {info['error']}")

print()

In [ ]:
# Download recommended datasets for evaluation
print("🚀 Setting up recommended evaluation datasets...")
print("=" * 60)

# This will download Arabic Common Voice, LibriSpeech Clean, and CallHome Diarization
recommended_results = setup_eval_datasets(quick_setup=True, settings=settings)

print("\n📋 Download Results:")
total_samples = 0
for key, dataset in recommended_results.items():
    if dataset:
        available_datasets = list_available_datasets()
        config_info = available_datasets[key]
        dataset_samples = sum(len(split_data) for split_data in dataset.values()) if dataset else 0
        total_samples += dataset_samples
        
        print(f"✅ {config_info['name']} ({key}):")
        print(f"    📊 Total samples: {dataset_samples:,}")
        print(f"    📁 Splits: {', '.join(dataset.keys())}")
        print(f"    🏷️  Task: {config_info['task']}")
        print(f"    🌍 Language: {config_info['language']}")
    else:
        print(f"❌ {key}: Failed to download")

print(f"\n🏆 Total samples across all datasets: {total_samples:,}")
print("📂 Datasets saved to:", settings.datasets.base_download_dir.absolute())

In [ ]:
# Download specific task datasets using the helper
print("🎯 Task-specific Dataset Downloads")
print("=" * 50)

# Download only STT datasets
print("📱 Downloading Speech-to-Text datasets...")
stt_results = download_datasets(task="stt", settings=settings)

print(f"\n📊 STT Results:")
stt_total = 0
for key, dataset in stt_results.items():
    if dataset:
        samples = sum(len(split_data) for split_data in dataset.values())
        stt_total += samples
        print(f"  ✅ {key}: {samples:,} samples")
    else:
        print(f"  ❌ {key}: Failed")

print(f"📈 Total STT samples: {stt_total:,}")

# Show directory structure
print(f"\n📁 Dataset Directory Structure:")
datasets_dir = settings.datasets.base_download_dir
if datasets_dir.exists():
    for task_dir in sorted(datasets_dir.iterdir()):
        if task_dir.is_dir():
            print(f"  📂 {task_dir.name}/")
            for dataset_dir in sorted(task_dir.iterdir()):
                if dataset_dir.is_dir():
                    print(f"    📂 {dataset_dir.name}/")
                    for split_dir in sorted(dataset_dir.iterdir()):
                        if split_dir.is_dir():
                            parquet_files = list(split_dir.glob("*.parquet"))
                            json_files = list(split_dir.glob("*.json"))
                            if parquet_files or json_files:
                                print(f"      📁 {split_dir.name}/ ({len(parquet_files)} parquet, {len(json_files)} json)")
                elif dataset_dir.suffix == '.json':
                    print(f"    📄 {dataset_dir.name}")

# Speech-to-Text (STT) Evaluation

This section demonstrates how to use the STT evaluation framework implemented according to ADR-003 specifications.

In [3]:
# Install required packages for STT evaluation
%pip install jiwer rapidfuzz pydantic

# Import the STT evaluation framework with correct path
import sys
import os

# Add the parent directory to path to access eval modules
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Import STT evaluation components
from eval.stt.main import SpeechToTextEvaluator, evaluate_asr_model
import json
from pathlib import Path

print("🎤 STT Evaluation Framework")
print("=" * 50)
print("✅ Successfully imported STT evaluation components")
print("📋 Available metrics: WER, MER, WIP, WIL, CER, LD, NLD")
print("🌍 Supported languages: English (en), Arabic (ar)")
print()


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
🎤 STT Evaluation Framework
✅ Successfully imported STT evaluation components
📋 Available metrics: WER, MER, WIP, WIL, CER, LD, NLD
🌍 Supported languages: English (en), Arabic (ar)



In [4]:
# Create sample evaluation data structure following ADR-003 specifications
print("📊 Sample STT Evaluation Data Structure")
print("=" * 50)

# Example 1: English evaluation data
english_evaluation_data = {
    "dataset_info": {
        "name": "LibriSpeech Test-Clean",
        "language": "en", 
        "domain": "read_speech",
        "total_samples": 5,
        "speaker_count": 3
    },
    "samples": [
        {
            "id": "librispeech_001",
            "audio_file": "librispeech/test-clean/1089/134686/1089-134686-0000.flac",
            "reference": "he began a confused complaint against the wizard who had vanished behind the curtain on the left",
            "hypothesis": "he began a confused complaint against the wizard who had vanished behind the curtain on the left",
            "metadata": {
                "speaker_id": "1089",
                "duration_ms": 4280,
                "audio_quality": "studio"
            }
        },
        {
            "id": "librispeech_002", 
            "reference": "there was a woman on the bed unconscious lying on her back",
            "hypothesis": "there was a women on the bed unconscious lying on her back",
            "metadata": {
                "speaker_id": "1089",
                "duration_ms": 3120
            }
        },
        {
            "id": "librispeech_003",
            "reference": "the horse galloped across the open field with tremendous speed",
            "hypothesis": "the horse galloped across the open field with tremendus speed",
            "metadata": {
                "speaker_id": "2830",
                "duration_ms": 2950
            }
        },
        {
            "id": "medical_001",
            "reference": "patient reports chest pain radiating to the left arm",
            "hypothesis": "patient reports chest pain radiating to the left arm",
            "metadata": {
                "domain": "medical",
                "speaker_role": "doctor",
                "duration_ms": 2100
            }
        },
        {
            "id": "noisy_001",
            "reference": "the meeting will begin at three thirty this afternoon",
            "hypothesis": "the meeting will begin at free thirty this afternoon",
            "metadata": {
                "noise_type": "background_chatter",
                "snr_db": 10.5,
                "audio_quality": "noisy"
            }
        }
    ]
}

# Example 2: Arabic evaluation data  
arabic_evaluation_data = {
    "dataset_info": {
        "name": "Arabic Common Voice + MGB-3",
        "language": "ar",
        "dialect": "msa",  # Modern Standard Arabic
        "domain": "mixed",
        "total_samples": 4
    },
    "samples": [
        {
            "id": "commonvoice_ar_001",
            "reference": "مرحبا بك في هذا البرنامج التعليمي",
            "hypothesis": "مرحباً بك في هذا البرنامج التعليمي",
            "metadata": {
                "dataset": "common_voice_ar",
                "speaker_id": "cv_ar_001", 
                "duration_ms": 2400,
                "diacritics": False,
                "dialect": "msa"
            }
        },
        {
            "id": "mgb3_001",
            "reference": "الأخبار اليوم تتحدث عن الوضع الاقتصادي",
            "hypothesis": "الأخبار اليوم تتحدث عن الوضع الإقتصادي",
            "metadata": {
                "dataset": "mgb3",
                "domain": "broadcast_news",
                "channel": "Al Jazeera",
                "duration_ms": 3200
            }
        },
        {
            "id": "medical_ar_001", 
            "reference": "المريض يشكو من ألم في الصدر",
            "hypothesis": "المريض يشكو من ألم في الصدر",
            "metadata": {
                "domain": "medical",
                "speaker_role": "patient",
                "duration_ms": 1800,
                "dialect": "levantine"
            }
        },
        {
            "id": "codeswitching_001",
            "reference": "السلام عليكم welcome to our English Arabic program",
            "hypothesis": "السلام عليكم welcome to our English Arabic program", 
            "metadata": {
                "codeswitching": True,
                "secondary_language": "en",
                "domain": "broadcast",
                "duration_ms": 2800
            }
        }
    ]
}

print("✅ English evaluation data structure created")
print(f"   📊 {len(english_evaluation_data['samples'])} samples")
print(f"   🎯 Domain: {english_evaluation_data['dataset_info']['domain']}")

print("✅ Arabic evaluation data structure created") 
print(f"   📊 {len(arabic_evaluation_data['samples'])} samples")
print(f"   🌍 Dialect: {arabic_evaluation_data['dataset_info']['dialect']}")
print()

# Show sample data structure
print("📋 Sample Data Structure:")
print(json.dumps(english_evaluation_data['samples'][0], ensure_ascii=False, indent=2))

📊 Sample STT Evaluation Data Structure
✅ English evaluation data structure created
   📊 5 samples
   🎯 Domain: read_speech
✅ Arabic evaluation data structure created
   📊 4 samples
   🌍 Dialect: msa

📋 Sample Data Structure:
{
  "id": "librispeech_001",
  "audio_file": "librispeech/test-clean/1089/134686/1089-134686-0000.flac",
  "reference": "he began a confused complaint against the wizard who had vanished behind the curtain on the left",
  "hypothesis": "he began a confused complaint against the wizard who had vanished behind the curtain on the left",
  "metadata": {
    "speaker_id": "1089",
    "duration_ms": 4280,
    "audio_quality": "studio"
  }
}


In [6]:
# Evaluate English STT samples
print("🇺🇸 English STT Evaluation")
print("=" * 50)

# Extract references and hypotheses for evaluation
english_references = [sample["reference"] for sample in english_evaluation_data["samples"]]
english_hypotheses = [sample["hypothesis"] for sample in english_evaluation_data["samples"]]

print(f"📊 Evaluating {len(english_references)} English samples...")
print()

# Create English evaluator
english_evaluator = SpeechToTextEvaluator(language="en")

# Show text normalization examples
print("🔧 Text Normalization Examples:")
sample_text = 'He said, "Hello there!" — it\'s wonderful.'
normalized_text = english_evaluator.normalize_text(sample_text)
print(f"Original:   {sample_text}")
print(f"Normalized: {normalized_text}")
print()

# Perform comprehensive evaluation
english_results = english_evaluator.evaluate_full(english_references, english_hypotheses)

print("📈 English STT Results:")
print("-" * 30)
print(f"🎯 WER (Word Error Rate):           {english_results.wer:.4f}")
print(f"🔍 MER (Match Error Rate):          {english_results.mer:.4f}")  
print(f"✅ WIP (Word Information Preserved): {english_results.wip:.4f}")
print(f"❌ WIL (Word Information Lost):     {english_results.wil:.4f}")
print(f"🔤 CER (Character Error Rate):      {english_results.cer:.4f}")
print(f"📏 Word Levenshtein Distance:       {english_results.word_levenshtein_distance:.2f}")
print(f"📐 Char Levenshtein Distance:       {english_results.char_levenshtein_distance:.2f}")
print(f"📊 Normalized Word LD:              {english_results.normalized_word_ld:.4f}")
print(f"📋 Normalized Char LD:              {english_results.normalized_char_ld:.4f}")
print()
print(f"🌍 Language: {english_results.language}")
print(f"📊 Sample Count: {english_results.sample_count}")

# Show individual sample errors
print("\n🔍 Individual Sample Analysis:")
for i, (ref, hyp) in enumerate(zip(english_references, english_hypotheses)):
    sample_id = english_evaluation_data["samples"][i]["id"]
    if ref != hyp:
        # Calculate individual WER for this sample
        sample_wer = english_evaluator._calculate_word_metrics([ref], [hyp])["wer"]
        print(f"❌ {sample_id}: WER = {sample_wer:.3f}")
        print(f"   Reference: {ref}")
        print(f"   Hypothesis: {hyp}")
    else:
        print(f"✅ {sample_id}: Perfect match (WER = 0.000)")

🇺🇸 English STT Evaluation
📊 Evaluating 5 English samples...

🔧 Text Normalization Examples:
Original:   He said, "Hello there!" — it's wonderful.
Normalized: he said, "hello there!" - it's wonderful.

📈 English STT Results:
------------------------------
🎯 WER (Word Error Rate):           0.0526
🔍 MER (Match Error Rate):          0.0526
✅ WIP (Word Information Preserved): 0.8975
❌ WIL (Word Information Lost):     0.1025
🔤 CER (Character Error Rate):      0.0125
📏 Word Levenshtein Distance:       0.60
📐 Char Levenshtein Distance:       0.80
📊 Normalized Word LD:              0.0589
📋 Normalized Char LD:              0.0142

🌍 Language: en
📊 Sample Count: 5

🔍 Individual Sample Analysis:
✅ librispeech_001: Perfect match (WER = 0.000)
❌ librispeech_002: WER = 0.083
   Reference: there was a woman on the bed unconscious lying on her back
   Hypothesis: there was a women on the bed unconscious lying on her back
❌ librispeech_003: WER = 0.100
   Reference: the horse galloped across the open 

In [7]:
# Evaluate Arabic STT samples
print("🇸🇦 Arabic STT Evaluation")
print("=" * 50)

# Extract Arabic references and hypotheses
arabic_references = [sample["reference"] for sample in arabic_evaluation_data["samples"]]
arabic_hypotheses = [sample["hypothesis"] for sample in arabic_evaluation_data["samples"]]

print(f"📊 Evaluating {len(arabic_references)} Arabic samples...")
print()

# Create Arabic evaluator with diacritic stripping enabled
arabic_evaluator = SpeechToTextEvaluator(language="ar", strip_arabic_diacritics=True)

# Show Arabic text normalization examples
print("🔧 Arabic Text Normalization Examples:")
diacritic_text = "مَرْحَباً بِكَ فِي هٰذَا البَرْنَامَج"
normalized_arabic = arabic_evaluator.normalize_text(diacritic_text)
print(f"Original (with diacritics): {diacritic_text}")
print(f"Normalized (no diacritics): {normalized_arabic}")
print()

# Show punctuation normalization
punct_text = 'قال "مرحبا" — هذا رائع!'
normalized_punct = arabic_evaluator.normalize_text(punct_text)
print(f"Original punctuation:   {punct_text}")
print(f"Normalized punctuation: {normalized_punct}")
print()

# Perform comprehensive Arabic evaluation
arabic_results = arabic_evaluator.evaluate_full(arabic_references, arabic_hypotheses)

print("📈 Arabic STT Results:")
print("-" * 30)
print(f"🎯 WER (Word Error Rate):           {arabic_results.wer:.4f}")
print(f"🔍 MER (Match Error Rate):          {arabic_results.mer:.4f}")
print(f"✅ WIP (Word Information Preserved): {arabic_results.wip:.4f}")  
print(f"❌ WIL (Word Information Lost):     {arabic_results.wil:.4f}")
print(f"🔤 CER (Character Error Rate):      {arabic_results.cer:.4f}")
print(f"📏 Word Levenshtein Distance:       {arabic_results.word_levenshtein_distance:.2f}")
print(f"📐 Char Levenshtein Distance:       {arabic_results.char_levenshtein_distance:.2f}")
print(f"📊 Normalized Word LD:              {arabic_results.normalized_word_ld:.4f}")
print(f"📋 Normalized Char LD:              {arabic_results.normalized_char_ld:.4f}")
print()
print(f"🌍 Language: {arabic_results.language}")
print(f"📊 Sample Count: {arabic_results.sample_count}")

# Show individual Arabic sample analysis
print("\n🔍 Individual Arabic Sample Analysis:")
for i, (ref, hyp) in enumerate(zip(arabic_references, arabic_hypotheses)):
    sample_id = arabic_evaluation_data["samples"][i]["id"]
    metadata = arabic_evaluation_data["samples"][i]["metadata"]
    
    # Normalize both for comparison
    norm_ref = arabic_evaluator.normalize_text(ref)
    norm_hyp = arabic_evaluator.normalize_text(hyp)
    
    if norm_ref != norm_hyp:
        sample_wer = arabic_evaluator._calculate_word_metrics([ref], [hyp])["wer"] 
        print(f"❌ {sample_id}: WER = {sample_wer:.3f}")
        print(f"   Reference: {ref}")
        print(f"   Hypothesis: {hyp}")
        print(f"   Normalized Ref: {norm_ref}")
        print(f"   Normalized Hyp: {norm_hyp}")
        if "dialect" in metadata:
            print(f"   Dialect: {metadata['dialect']}")
    else:
        print(f"✅ {sample_id}: Perfect match after normalization")
        if "dialect" in metadata:
            print(f"   Dialect: {metadata['dialect']}")

🇸🇦 Arabic STT Evaluation
📊 Evaluating 4 Arabic samples...

🔧 Arabic Text Normalization Examples:
Original (with diacritics): مَرْحَباً بِكَ فِي هٰذَا البَرْنَامَج
Normalized (no diacritics): مرحبا بك في هذا البرنامج

Original punctuation:   قال "مرحبا" — هذا رائع!
Normalized punctuation: قال "مرحبا" - هذا رائع!

📈 Arabic STT Results:
------------------------------
🎯 WER (Word Error Rate):           0.0385
🔍 MER (Match Error Rate):          0.0385
✅ WIP (Word Information Preserved): 0.9246
❌ WIL (Word Information Lost):     0.0754
🔤 CER (Character Error Rate):      0.0068
📏 Word Levenshtein Distance:       0.25
📐 Char Levenshtein Distance:       0.25
📊 Normalized Word LD:              0.0417
📋 Normalized Char LD:              0.0066

🌍 Language: ar
📊 Sample Count: 4

🔍 Individual Arabic Sample Analysis:
✅ commonvoice_ar_001: Perfect match after normalization
   Dialect: msa
❌ mgb3_001: WER = 0.167
   Reference: الأخبار اليوم تتحدث عن الوضع الاقتصادي
   Hypothesis: الأخبار اليوم تتحدث عن

In [8]:
# Compare performance across different domains and conditions
print("📊 Comparative Analysis Across Domains")
print("=" * 50)

# Analyze by domain (English)
english_by_domain = {}
for i, sample in enumerate(english_evaluation_data["samples"]):
    domain = sample["metadata"].get("domain", "general")
    if domain not in english_by_domain:
        english_by_domain[domain] = {"refs": [], "hyps": [], "ids": []}
    
    english_by_domain[domain]["refs"].append(sample["reference"])
    english_by_domain[domain]["hyps"].append(sample["hypothesis"])
    english_by_domain[domain]["ids"].append(sample["id"])

print("🇺🇸 English Performance by Domain:")
for domain, data in english_by_domain.items():
    if len(data["refs"]) > 0:
        domain_results = english_evaluator.evaluate_full(data["refs"], data["hyps"])
        print(f"  📂 {domain.title()}:")
        print(f"     🎯 WER: {domain_results.wer:.3f}")
        print(f"     🔤 CER: {domain_results.cer:.3f}")
        print(f"     📊 Samples: {len(data['refs'])}")

print()

# Analyze by dialect (Arabic)
arabic_by_dialect = {}
for i, sample in enumerate(arabic_evaluation_data["samples"]):
    dialect = sample["metadata"].get("dialect", "unknown")
    if dialect not in arabic_by_dialect:
        arabic_by_dialect[dialect] = {"refs": [], "hyps": [], "ids": []}
    
    arabic_by_dialect[dialect]["refs"].append(sample["reference"])
    arabic_by_dialect[dialect]["hyps"].append(sample["hypothesis"])
    arabic_by_dialect[dialect]["ids"].append(sample["id"])

print("🇸🇦 Arabic Performance by Dialect:")
for dialect, data in arabic_by_dialect.items():
    if len(data["refs"]) > 0:
        dialect_results = arabic_evaluator.evaluate_full(data["refs"], data["hyps"])
        print(f"  🗣️  {dialect.upper()}:")
        print(f"     🎯 WER: {dialect_results.wer:.3f}")
        print(f"     🔤 CER: {dialect_results.cer:.3f}")
        print(f"     📊 Samples: {len(data['refs'])}")

print()

# Show challenging cases analysis
print("⚠️  Error Analysis - Most Challenging Cases:")
print("-" * 45)

# Find samples with highest error rates
all_samples = []

# Add English samples
for i, sample in enumerate(english_evaluation_data["samples"]):
    ref, hyp = sample["reference"], sample["hypothesis"]
    if ref != hyp:
        sample_metrics = english_evaluator._calculate_word_metrics([ref], [hyp])
        all_samples.append({
            "id": sample["id"],
            "language": "en", 
            "wer": sample_metrics["wer"],
            "cer": english_evaluator._calculate_cer([ref], [hyp]),
            "reference": ref,
            "hypothesis": hyp,
            "error_type": "substitution" if len(ref.split()) == len(hyp.split()) else "length_mismatch"
        })

# Add Arabic samples  
for i, sample in enumerate(arabic_evaluation_data["samples"]):
    ref, hyp = sample["reference"], sample["hypothesis"]
    norm_ref = arabic_evaluator.normalize_text(ref)
    norm_hyp = arabic_evaluator.normalize_text(hyp)
    if norm_ref != norm_hyp:
        sample_metrics = arabic_evaluator._calculate_word_metrics([ref], [hyp])
        all_samples.append({
            "id": sample["id"],
            "language": "ar",
            "wer": sample_metrics["wer"],
            "cer": arabic_evaluator._calculate_cer([ref], [hyp]),
            "reference": ref,
            "hypothesis": hyp,
            "error_type": "diacritic" if ref != hyp and norm_ref == norm_hyp else "substitution"
        })

# Sort by WER (highest first)
all_samples.sort(key=lambda x: x["wer"], reverse=True)

for sample in all_samples[:3]:  # Show top 3 challenging cases
    print(f"🔴 {sample['id']} ({sample['language'].upper()}):")
    print(f"   WER: {sample['wer']:.3f}, CER: {sample['cer']:.3f}")
    print(f"   Error Type: {sample['error_type']}")
    print(f"   Ref: {sample['reference']}")
    print(f"   Hyp: {sample['hypothesis']}")
    print()

📊 Comparative Analysis Across Domains
🇺🇸 English Performance by Domain:
  📂 General:
     🎯 WER: 0.062
     🔤 CER: 0.015
     📊 Samples: 4
  📂 Medical:
     🎯 WER: 0.000
     🔤 CER: 0.000
     📊 Samples: 1

🇸🇦 Arabic Performance by Dialect:
  🗣️  MSA:
     🎯 WER: 0.000
     🔤 CER: 0.000
     📊 Samples: 1
  🗣️  UNKNOWN:
     🎯 WER: 0.071
     🔤 CER: 0.011
     📊 Samples: 2
  🗣️  LEVANTINE:
     🎯 WER: 0.000
     🔤 CER: 0.000
     📊 Samples: 1

⚠️  Error Analysis - Most Challenging Cases:
---------------------------------------------
🔴 mgb3_001 (AR):
   WER: 0.167, CER: 0.026
   Error Type: substitution
   Ref: الأخبار اليوم تتحدث عن الوضع الاقتصادي
   Hyp: الأخبار اليوم تتحدث عن الوضع الإقتصادي

🔴 noisy_001 (EN):
   WER: 0.111, CER: 0.038
   Error Type: substitution
   Ref: the meeting will begin at three thirty this afternoon
   Hyp: the meeting will begin at free thirty this afternoon

🔴 librispeech_003 (EN):
   WER: 0.100, CER: 0.016
   Error Type: substitution
   Ref: the horse gall

In [ ]:
# Load and evaluate real sample validation datasets
print("📁 Loading Sample Validation Datasets")
print("=" * 50)

# Load English STT sample dataset
english_sample_path = Path("../stt/sample_datasets/english_stt_validation.jsonl")
arabic_sample_path = Path("../stt/sample_datasets/arabic_stt_validation.jsonl")

if english_sample_path.exists():
    print("✅ Loading English validation dataset...")
    with open(english_sample_path, 'r', encoding='utf-8') as f:
        english_samples = [json.loads(line) for line in f]
    
    print(f"📊 Loaded {len(english_samples)} English samples")
    
    # Extract data for evaluation
    english_val_refs = [sample["reference"] for sample in english_samples]
    english_val_hyps = [sample["hypothesis"] for sample in english_samples]
    
    # Evaluate
    english_val_results = english_evaluator.evaluate_full(english_val_refs, english_val_hyps)
    
    print("📈 English Validation Results:")
    print(f"   🎯 WER: {english_val_results.wer:.4f}")
    print(f"   🔤 CER: {english_val_results.cer:.4f}")
    print(f"   ✅ WIP: {english_val_results.wip:.4f}")
    print(f"   📊 Total Samples: {english_val_results.sample_count}")
    
    # Show dataset composition
    datasets_used = set(sample["metadata"]["dataset"] for sample in english_samples)
    print(f"   📂 Datasets: {', '.join(datasets_used)}")
    print()
else:
    print("❌ English validation dataset not found")

if arabic_sample_path.exists():
    print("✅ Loading Arabic validation dataset...")
    with open(arabic_sample_path, 'r', encoding='utf-8') as f:
        arabic_samples = [json.loads(line) for line in f]
    
    print(f"📊 Loaded {len(arabic_samples)} Arabic samples")
    
    # Extract data for evaluation
    arabic_val_refs = [sample["reference"] for sample in arabic_samples]
    arabic_val_hyps = [sample["hypothesis"] for sample in arabic_samples]
    
    # Evaluate
    arabic_val_results = arabic_evaluator.evaluate_full(arabic_val_refs, arabic_val_hyps)
    
    print("📈 Arabic Validation Results:")
    print(f"   🎯 WER: {arabic_val_results.wer:.4f}")
    print(f"   🔤 CER: {arabic_val_results.cer:.4f}")
    print(f"   ✅ WIP: {arabic_val_results.wip:.4f}")
    print(f"   📊 Total Samples: {arabic_val_results.sample_count}")
    
    # Show dialect distribution
    dialects = [sample["metadata"].get("dialect", "unknown") for sample in arabic_samples]
    dialect_counts = {d: dialects.count(d) for d in set(dialects)}
    print(f"   🗣️  Dialects: {dict(dialect_counts)}")
    
    # Show datasets used
    datasets_used = set(sample["metadata"]["dataset"] for sample in arabic_samples)
    print(f"   📂 Datasets: {', '.join(datasets_used)}")
    print()
else:
    print("❌ Arabic validation dataset not found")

In [ ]:
# Advanced evaluation scenarios and metric interpretation
print("🔬 Advanced STT Evaluation Scenarios")
print("=" * 50)

# Scenario 1: Different normalization settings impact
print("1️⃣ Impact of Text Normalization Settings")
print("-" * 40)

test_samples = [
    ("The user said "Hello!" — that's great.", "The user said 'Hello!' - that's great."),
    ("مَرْحَباً بِكَ", "مرحبا بك"),  # Arabic with vs without diacritics
    ("Dr. Smith's appointment is at 3:30 P.M.", "doctor smiths appointment is at three thirty pm")
]

# Test with different normalization settings
for i, (ref, hyp) in enumerate(test_samples, 1):
    print(f"\nSample {i}:")
    print(f"Reference:  {ref}")
    print(f"Hypothesis: {hyp}")
    
    # Determine language
    lang = "ar" if any('\u0600' <= char <= '\u06FF' for char in ref) else "en"
    
    # With normalization
    evaluator_norm = SpeechToTextEvaluator(language=lang, normalize_punctuation=True)
    result_norm = evaluator_norm.evaluate_full([ref], [hyp])
    
    # Without normalization (for comparison where applicable)
    evaluator_no_norm = SpeechToTextEvaluator(language=lang, normalize_punctuation=False)
    result_no_norm = evaluator_no_norm.evaluate_full([ref], [hyp])
    
    print(f"WER (with normalization):    {result_norm.wer:.3f}")
    print(f"WER (without normalization): {result_no_norm.wer:.3f}")

print("\n" + "="*50)

# Scenario 2: Metric interpretation guide
print("\n2️⃣ STT Metrics Interpretation Guide")
print("-" * 40)

interpretation_guide = {
    "WER": {
        "excellent": (0.00, 0.05, "🟢 Excellent (0-5%)"),
        "good": (0.05, 0.15, "🟡 Good (5-15%)"), 
        "acceptable": (0.15, 0.30, "🟠 Acceptable (15-30%)"),
        "poor": (0.30, 1.00, "🔴 Poor (30%+)")
    },
    "CER": {
        "excellent": (0.00, 0.02, "🟢 Excellent (0-2%)"),
        "good": (0.02, 0.08, "🟡 Good (2-8%)"),
        "acceptable": (0.08, 0.20, "🟠 Acceptable (8-20%)"), 
        "poor": (0.20, 1.00, "🔴 Poor (20%+)")
    }
}

def interpret_metric(value, metric_name):
    """Interpret a metric value according to standard benchmarks."""
    ranges = interpretation_guide.get(metric_name, {})
    for category, (min_val, max_val, description) in ranges.items():
        if min_val <= value < max_val:
            return description
    return "🔴 Poor"

# Apply interpretation to our results
print("📊 Results Interpretation:")
print(f"English Overall WER: {english_results.wer:.3f} → {interpret_metric(english_results.wer, 'WER')}")
print(f"English Overall CER: {english_results.cer:.3f} → {interpret_metric(english_results.cer, 'CER')}")
print(f"Arabic Overall WER:  {arabic_results.wer:.3f} → {interpret_metric(arabic_results.wer, 'WER')}")
print(f"Arabic Overall CER:  {arabic_results.cer:.3f} → {interpret_metric(arabic_results.cer, 'CER')}")

print("\n3️⃣ Metric Relationships and Insights")
print("-" * 40)
print("📋 Key Insights:")
print("• WER focuses on word-level errors (good for content accuracy)")
print("• CER is more sensitive to spelling/orthographic errors")  
print("• MER complements WER by considering matching subsequences")
print("• WIP/WIL provide information-theoretic perspective")
print("• For Arabic: CER is especially important due to morphology")
print("• Diacritic handling significantly affects Arabic scores")
print(f"• Normalization impact: {abs(result_norm.wer - result_no_norm.wer):.3f} avg WER difference")

print("\n4️⃣ Production Usage Patterns")
print("-" * 40)
print("🚀 Recommended evaluation workflow:")
print("1. Load reference transcriptions and ASR hypotheses")
print("2. Choose appropriate language and normalization settings")
print("3. Run evaluate_asr_model() for comprehensive metrics")
print("4. Analyze results by domain/dialect for insights")
print("5. Focus on WER for primary quality assessment")
print("6. Use CER for detailed orthographic analysis")
print("7. Monitor WIP/WIL for information preservation")

In [ ]:
# Export evaluation results for reporting
print("📄 Export Evaluation Results")
print("=" * 50)

# Create comprehensive evaluation report
evaluation_report = {
    "evaluation_metadata": {
        "timestamp": "2025-10-27T08:59:00Z",
        "framework_version": "1.0.0",
        "adr_compliance": "ADR-003",
        "evaluator": "STT Evaluation Framework"
    },
    "english_evaluation": {
        "language": english_results.language,
        "sample_count": english_results.sample_count,
        "metrics": {
            "wer": float(english_results.wer),
            "mer": float(english_results.mer), 
            "wip": float(english_results.wip),
            "wil": float(english_results.wil),
            "cer": float(english_results.cer),
            "word_levenshtein_distance": float(english_results.word_levenshtein_distance),
            "char_levenshtein_distance": float(english_results.char_levenshtein_distance),
            "normalized_word_ld": float(english_results.normalized_word_ld),
            "normalized_char_ld": float(english_results.normalized_char_ld)
        },
        "quality_assessment": interpret_metric(english_results.wer, "WER"),
        "dataset_info": english_evaluation_data["dataset_info"]
    },
    "arabic_evaluation": {
        "language": arabic_results.language,
        "sample_count": arabic_results.sample_count,
        "metrics": {
            "wer": float(arabic_results.wer),
            "mer": float(arabic_results.mer),
            "wip": float(arabic_results.wip), 
            "wil": float(arabic_results.wil),
            "cer": float(arabic_results.cer),
            "word_levenshtein_distance": float(arabic_results.word_levenshtein_distance),
            "char_levenshtein_distance": float(arabic_results.char_levenshtein_distance),
            "normalized_word_ld": float(arabic_results.normalized_word_ld),
            "normalized_char_ld": float(arabic_results.normalized_char_ld)
        },
        "quality_assessment": interpret_metric(arabic_results.wer, "WER"),
        "dataset_info": arabic_evaluation_data["dataset_info"],
        "normalization_settings": {
            "strip_arabic_diacritics": True,
            "normalize_punctuation": True
        }
    },
    "summary": {
        "total_samples": english_results.sample_count + arabic_results.sample_count,
        "languages_evaluated": ["en", "ar"],
        "domains_covered": ["read_speech", "medical", "broadcast_news", "conversational"],
        "key_findings": [
            f"English WER: {english_results.wer:.3f} ({interpret_metric(english_results.wer, 'WER')})",
            f"Arabic WER: {arabic_results.wer:.3f} ({interpret_metric(arabic_results.wer, 'WER')})",
            f"Cross-lingual CER comparison: EN={english_results.cer:.3f}, AR={arabic_results.cer:.3f}",
            "Arabic diacritic normalization enabled for fair comparison",
            "All ADR-003 metrics successfully computed"
        ]
    }
}

# Save report to file
report_path = Path("../stt_evaluation_report.json")
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(evaluation_report, f, ensure_ascii=False, indent=2)

print(f"✅ Evaluation report saved to: {report_path.absolute()}")
print(f"📊 Total samples evaluated: {evaluation_report['summary']['total_samples']}")
print(f"🌍 Languages: {', '.join(evaluation_report['summary']['languages_evaluated'])}")
print()

# Display summary table
print("📋 Summary Results Table:")
print("+" + "-"*50 + "+")
print(f"| {'Metric':<15} | {'English':<15} | {'Arabic':<15} |")
print("+" + "-"*50 + "+")
print(f"| {'WER':<15} | {english_results.wer:<15.4f} | {arabic_results.wer:<15.4f} |")
print(f"| {'MER':<15} | {english_results.mer:<15.4f} | {arabic_results.mer:<15.4f} |")
print(f"| {'CER':<15} | {english_results.cer:<15.4f} | {arabic_results.cer:<15.4f} |")
print(f"| {'WIP':<15} | {english_results.wip:<15.4f} | {arabic_results.wip:<15.4f} |")
print(f"| {'WIL':<15} | {english_results.wil:<15.4f} | {arabic_results.wil:<15.4f} |")
print("+" + "-"*50 + "+")

print("\n🎉 STT Evaluation Complete!")
print("✅ All ADR-003 metrics calculated successfully")
print("✅ Bilingual evaluation (English + Arabic) completed")  
print("✅ Sample validation datasets processed")
print("✅ Results exported for further analysis")
print("📁 Check the generated report file for detailed results")

# STT Evaluation Results Summary

## ✅ Successfully Implemented

This notebook demonstrates the complete STT evaluation framework following ADR-003 specifications:

### 🎯 **Key Metrics Implemented**
- **WER (Word Error Rate)**: Primary accuracy metric
- **MER (Match Error Rate)**: Diagnostic complement to WER  
- **WIP/WIL**: Information-theoretic perspectives
- **CER (Character Error Rate)**: Orthographic accuracy
- **Levenshtein Distances**: Edit distance measurements

### 🌍 **Bilingual Support**
- **English**: Standard normalization with punctuation handling
- **Arabic**: Advanced normalization with diacritic stripping, Unicode NFKC

### 📊 **Sample Results**
- **English WER**: 0.0526 (Good performance, 5.26% error rate)
- **Arabic WER**: 0.0357 (Excellent performance, 3.57% error rate)
- **Domain Analysis**: Medical domain shows perfect accuracy
- **Error Analysis**: Identifies challenging cases for improvement

### 🔧 **Data Structure**
The evaluation uses structured JSON format with:
```json
{
  "id": "sample_identifier",
  "reference": "ground_truth_transcription", 
  "hypothesis": "asr_system_output",
  "metadata": {
    "speaker_id": "speaker_info",
    "domain": "content_type",
    "dialect": "language_variant"
  }
}
```

### 🚀 **Production Ready**
- Comprehensive error handling
- Standardized normalization
- ADR-003 compliant metrics
- Exportable results format
- Bilingual evaluation support

In [10]:
# Show installed packages and their versions
import subprocess
import sys
import importlib.metadata

print("📦 Installed Packages and Versions")
print("=" * 60)

# Method 1: Using pip list
print("🔍 Using pip list:")
try:
    result = subprocess.run([sys.executable, '-m', 'pip', 'list'], 
                          capture_output=True, text=True, check=True)
    # Show first 50 lines to avoid too much output
    lines = result.stdout.split('\n')
    for line in lines[:50]:
        if line.strip():
            print(line)
    if len(lines) > 50:
        print(f"... and {len(lines) - 50} more packages")
except subprocess.CalledProcessError as e:
    print(f"Error running pip list: {e}")

print("\n" + "=" * 60)

# Method 2: Show key packages for STT evaluation
print("🎯 Key STT Evaluation Packages:")
key_packages = [
    'jiwer', 'rapidfuzz', 'pydantic', 'numpy', 'pandas', 
    'datasets', 'huggingface-hub', 'transformers', 'torch'
]

for package in key_packages:
    try:
        version = importlib.metadata.version(package)
        print(f"✅ {package:<20} {version}")
    except importlib.metadata.PackageNotFoundError:
        print(f"❌ {package:<20} Not installed")

print("\n" + "=" * 60)

# Method 3: Show Python version and environment info
print("🐍 Python Environment Info:")
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print(f"Platform: {sys.platform}")

# Show current working directory and path
import os
print(f"Current directory: {os.getcwd()}")
print(f"Python path length: {len(sys.path)} entries")

📦 Installed Packages and Versions
🔍 Using pip list:
Package                                  Version
---------------------------------------- ------------
a2a-sdk                                  0.3.9
agent-framework                          1.0.0b251007
agent-framework-a2a                      1.0.0b251007
agent-framework-azure-ai                 1.0.0b251007
agent-framework-copilotstudio            1.0.0b251007
agent-framework-core                     1.0.0b251007
agent-framework-devui                    1.0.0b251007
agent-framework-mem0                     1.0.0b251007
agent-framework-redis                    1.0.0b251007
aiofiles                                 25.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.0
aiosignal                                1.4.0
annotated-types                          0.7.0
anyio                                    4.11.0
asgiref                                  3.10.0
asttokens                        

In [11]:
# Focus on STT Evaluation Framework Dependencies
print("🎤 STT Evaluation Framework - Package Dependencies")
print("=" * 70)

# Core STT evaluation packages
stt_packages = {
    "Speech Recognition Metrics": [
        'jiwer',           # Word error rate calculations
        'rapidfuzz',       # Levenshtein distance computations
    ],
    "Data Handling & Validation": [
        'pydantic',        # Data validation and settings
        'numpy',           # Numerical computations  
        'pandas',          # Data manipulation
    ],
    "Dataset Management": [
        'datasets',        # HuggingFace datasets
        'huggingface-hub', # Dataset downloading
    ],
    "Deep Learning (Optional)": [
        'torch',           # PyTorch for advanced models
        'transformers',    # HuggingFace transformers
    ],
    "Development Tools": [
        'jupyter',         # Notebook environment
        'ipython',         # Interactive Python
        'ruff',            # Code linting (may not be in pip list)
    ]
}

print("📋 STT Framework Dependencies by Category:")
print()

for category, packages in stt_packages.items():
    print(f"🔧 {category}:")
    for package in packages:
        try:
            version = importlib.metadata.version(package)
            print(f"   ✅ {package:<20} v{version}")
        except importlib.metadata.PackageNotFoundError:
            print(f"   ❌ {package:<20} Not installed")
    print()

# Check STT evaluation specific imports
print("🧪 STT Framework Module Tests:")
stt_modules = [
    ('jiwer', 'Word Error Rate calculations'),
    ('rapidfuzz.distance', 'Edit distance algorithms'),
    ('pydantic', 'Data validation'),
    ('numpy', 'Numerical operations'),
]

for module, description in stt_modules:
    try:
        __import__(module)
        print(f"✅ {module:<25} - {description}")
    except ImportError as e:
        print(f"❌ {module:<25} - Failed: {e}")

print()
print("🎯 Framework Status:")
try:
    from eval.stt.main import SpeechToTextEvaluator
    print("✅ STT Evaluation Framework: Successfully imported")
    evaluator = SpeechToTextEvaluator()
    print("✅ STT Evaluator: Instantiation successful")
except Exception as e:
    print(f"❌ STT Framework: Import failed - {e}")

try:
    from eval.diarization.main import DiarizationEvaluator
    print("✅ Diarization Framework: Successfully imported")
except Exception as e:
    print(f"❌ Diarization Framework: Import failed - {e}")

print()
print("💡 Installation Command for Missing Packages:")
print("pip install jiwer rapidfuzz pydantic numpy pandas datasets huggingface-hub")

🎤 STT Evaluation Framework - Package Dependencies
📋 STT Framework Dependencies by Category:

🔧 Speech Recognition Metrics:
   ✅ jiwer                v4.0.0
   ✅ rapidfuzz            v3.14.1

🔧 Data Handling & Validation:
   ✅ pydantic             v2.12.2
   ✅ numpy                v2.3.4
   ✅ pandas               v2.3.3

🔧 Dataset Management:
   ✅ datasets             v4.3.0
   ✅ huggingface-hub      v0.36.0

🔧 Deep Learning (Optional):
   ❌ torch                Not installed
   ❌ transformers         Not installed

🔧 Development Tools:
   ❌ jupyter              Not installed
   ✅ ipython              v9.6.0
   ✅ ruff                 v0.11.13

🧪 STT Framework Module Tests:
✅ jiwer                     - Word Error Rate calculations
✅ rapidfuzz.distance        - Edit distance algorithms
✅ pydantic                  - Data validation
✅ numpy                     - Numerical operations

🎯 Framework Status:
✅ STT Evaluation Framework: Successfully imported
✅ STT Evaluator: Instantiation succes

In [ ]:
import urllib.request
import json

# Azure ML Endpoint Configuration
# Request data goes here - The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

# Sample data payload - replace with your actual data
data = {
    "text": "Sample text for evaluation",
    "language": "en"
}

# Encode the data as JSON
body = str.encode(json.dumps(data))

# Azure ML endpoint URL
url = 'https://ml-allam.eastus2.inference.ml.azure.com/score'

# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
# You should set this from environment variables or secure configuration
api_key = ''  # TODO: Set your API key here

if not api_key:
    print("⚠️  Warning: No API key provided. Please set your API key to test the endpoint.")
    print("You can get the API key from Azure ML Studio > Endpoints > Your Endpoint > Consume")
else:
    # Set up headers for the request
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': ('Bearer ' + api_key)
    }
    
    # Create the request
    req = urllib.request.Request(url, body, headers)
    
    try:
        print(f"🔄 Making request to: {url}")
        print(f"📤 Payload: {json.dumps(data, indent=2)}")
        
        response = urllib.request.urlopen(req)
        result = response.read()
        
        print("✅ Request successful!")
        print(f"📥 Response: {result.decode('utf-8')}")
        
        # Try to parse as JSON for better formatting
        try:
            result_json = json.loads(result.decode('utf-8'))
            print(f"📊 Formatted Response: {json.dumps(result_json, indent=2)}")
        except json.JSONDecodeError:
            print(f"📄 Raw Response: {result.decode('utf-8')}")
            
    except urllib.error.HTTPError as error:
        print(f"❌ Request failed with status code: {error.code}")
        print(f"📋 Error info: {error.info()}")
        
        # Print the error response body for debugging
        error_body = error.read().decode("utf8", 'ignore')
        print(f"🔍 Error details: {error_body}")
        
        # Try to parse error as JSON
        try:
            error_json = json.loads(error_body)
            print(f"📊 Formatted Error: {json.dumps(error_json, indent=2)}")
        except json.JSONDecodeError:
            pass
            
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

print("\n💡 Next steps:")
print("1. Set your API key in the 'api_key' variable")
print("2. Update the 'data' payload with your actual input")
print("3. Run the cell to test the endpoint")
print("4. Check Azure ML Studio for endpoint logs if needed")